In [1]:
import re
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pymongo
import json

# Connect to Mongo and Get Databases

mongo_connect_string = 'mongodb://TwitterIPO?authSource=TwitterIPO'

client = pymongo.MongoClient(mongo_connect_string)
db = client.TwitterIPO
RawTweets = db.RawTweets
ProcessedTweets = db.ProcessedTweets
PriceData = db.PriceData
Vader = db.Vader
MNB_Logit_SVM_Sentiment = db.MNB_Logit_SVM_Sentiment2
TweetWordCount = db.TweetWordCount
FinalDataset = db.FinalDataset

In [2]:
# Get Price Date and Create a Dataframe
prices = [x for x in PriceData.find()]
df_prices = pd.DataFrame(prices)
df_prices['Date'] = pd.to_datetime(df_prices.Date)
# Format the date
df_prices['Date'] = df_prices.Date.apply(lambda x: datetime.utcfromtimestamp(x.value / 1e9).date())
df_prices.head()

,Adj_Close,Adj_High,Adj_Low,Adj_Open,Adj_Volume,Close,Date,Dividend,High,Low,Name,Open,Split,UID,Volume,_id,class_num_label,direction,return
0,19.89,20.69,19.75,20.65,25630157.0,19.89,2017-03-16,0.0,20.69,19.75,SNAP,20.65,1.0,SNAP10,25630157.0,58efeb8b1845123420836caf,0,Down,-0.042369
1,21.82,22.25,20.52,20.65,47599301.0,21.82,2017-03-22,0.0,22.25,20.52,SNAP,20.65,1.0,SNAP14,47599301.0,58efeb8b1845123420836cb0,1,Up,0.070658
2,22.81,23.43,21.31,22.03,49834423.0,22.81,2017-03-08,0.0,23.43,21.31,SNAP,22.03,1.0,SNAP4,49834423.0,58efeb8b1845123420836cb1,1,Up,0.063899
3,20.38,20.54,19.55,20.04,20080142.0,20.38,2017-03-21,0.0,20.54,19.55,SNAP,20.04,1.0,SNAP13,20080142.0,58efeb8b1845123420836cb2,1,Up,0.022579
4,21.44,22.50,20.64,22.21,71849684.0,21.44,2017-03-07,0.0,22.50,20.64,SNAP,22.21,1.0,SNAP3,71849684.0,58efeb8b1845123420836cb3,0,Down,-0.098023


In [3]:
# Get Twitter Data
df_tweets = pd.DataFrame(list(ProcessedTweets.find({'lang' : 'en'})))
df_tweets.head()
print(df_tweets.shape)

(23749, 8)


In [4]:
# Get Sentiment Data

In [5]:
df_MNBLogSVM = pd.DataFrame(list(MNB_Logit_SVM_Sentiment.find()))
df_MNBLogSVM.shape

(23749, 5)

In [6]:
# Merge MNBLogSVM Data with Twitter Data
df_senti = pd.merge(df_tweets, df_MNBLogSVM, on='UID',how='inner')

In [7]:
# Convert Merged Date to Date format for joining
df_senti.datepy = df_senti.datepy.map(lambda x: x.date())

In [8]:
# Map into Positive and Negative
df_senti['logit_label'] = df_senti.logit_predict.map({1 : 'Positive', 0 : 'Negative'})
df_senti['mnb_label'] = df_senti.mnb_predict.map({1 : 'Positive', 0 : 'Negative'})
df_senti['svm_label'] = df_senti.svm_predict.map({1 : 'Positive', 0 : 'Negative'})

In [9]:
# Keep columns that we want. Get rid of the rest. Divide into Seperate Tables for SVM, LOGIT & MNB for easier joining.

df_svm = df_senti[['company', 'datepy', 'svm_label']]
df_mnb = df_senti[['company', 'datepy', 'mnb_label']]
df_logit = df_senti[['company', 'datepy', 'logit_label']]

# Get Counts of each

svm_counts = df_svm.groupby(by=['company','datepy'])
svm_counts = svm_counts.svm_label.value_counts()

mnb_counts = df_mnb.groupby(by=['company','datepy'])
mnb_counts = mnb_counts.mnb_label.value_counts()

logit_counts = df_logit.groupby(by=['company','datepy'])
logit_counts = logit_counts.logit_label.value_counts()

# Turn into dicts for joining

svm_dict = []
for k,v in svm_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['SVM_Pos_Count'] = v
    else:
        s['SVM_Neg_Count'] = v

    svm_dict.append(s)
    
mnb_dict = []
for k,v in mnb_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['MNB_Pos_Count'] = v
    else:
        s['MNB_Neg_Count'] = v

    mnb_dict.append(s)
    
logit_dict = []
for k,v in logit_counts.iteritems():
    
    log_res = {}
    
    log_res['Name'] = k[0]
    
    log_res['Date'] = k[1]
    
    if k[2] == 'Positive':
        log_res['Logit_Pos_Count'] = v
    else:
        log_res['Logit_Neg_Count'] = v

    logit_dict.append(log_res)

    
# Convert back to dataframes and reset index

svm_counts = pd.DataFrame(svm_dict)
svm_counts = svm_counts.groupby(['Name','Date']).sum()
svm_counts = svm_counts.fillna(0)
svm_counts = svm_counts.reset_index(level=['Date','Name'])

mnb_counts = pd.DataFrame(mnb_dict)
mnb_counts = mnb_counts.groupby(['Name','Date']).sum()
mnb_counts = mnb_counts.fillna(0)
mnb_counts = mnb_counts.reset_index(level=['Date','Name'])

logit_counts = pd.DataFrame(logit_dict)
logit_counts = logit_counts.groupby(['Name','Date']).sum()
logit_counts = logit_counts.fillna(0)
logit_counts = logit_counts.reset_index(level=['Date','Name'])

<h2> TweetWordCount - Average Ratio & Counts </h2>

In [10]:
# Get Tweet Word Count and merge with Processed Tweets
df_tweetwordcount = pd.DataFrame(list(TweetWordCount.find()))
df_tweetwordcount = pd.merge(df_tweets, df_tweetwordcount, on='UID',how='inner')

# Convert Datetime object to date
df_tweetwordcount.datepy = df_tweetwordcount.datepy.map(lambda x: x.date())

# Add Labels
df_tweetwordcount['twc_label'] = np.where(df_tweetwordcount['ratio'] > 0, 'Positive','Negative')
df_tweetwordcount.loc[df_tweetwordcount.ratio == 0, 'twc_label'] = 'Neutral'

# Groupby Date and Company for Average Ratio
df_tweetwordcount_ratio = df_tweetwordcount.groupby(['datepy','company']).mean()

# Keep Date, Company & Ratio
df_tweetwordcount_ratio = df_tweetwordcount_ratio[['ratio']]
df_tweetwordcount_ratio = df_tweetwordcount_ratio.reset_index(['datepy','company'])
df_tweetwordcount_ratio = df_tweetwordcount_ratio.rename(columns={'datepy' : 'Date','company':'Name','ratio' : 'WordCountRatio'})

# Get Counts

df_tweetwordcount_counts = df_tweetwordcount[['company', 'datepy', 'twc_label']]

twc_counts = df_tweetwordcount_counts.groupby(by=['company','datepy'])
twc_counts = twc_counts.twc_label.value_counts()

# Turn counts into DF for easier conversion to pandas object
twc_dict = []
for k,v in twc_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['TWC_Pos_Count'] = v
    elif k[2] == 'Neutral':
        s['TWC_Neutral_Count'] = v
    else:
        s['TWC_Neg_Count'] = v

    twc_dict.append(s)

# Turn TWC Dict to Dataframe

twc_counts = pd.DataFrame(twc_dict)
twc_counts = twc_counts.groupby(['Name','Date']).sum()
twc_counts = twc_counts.fillna(0)
twc_counts = twc_counts.reset_index(level=['Date','Name'])

<h2> Vader Incorporation </h2>

In [11]:
# Get Vader and merge with Processed Tweets
df_vader = pd.DataFrame(list(Vader.find()))
df_vader = pd.merge(df_tweets, df_vader, on='UID',how='inner')

# Convert Datetime object to date
df_vader.datepy = df_vader.datepy.map(lambda x: x.date())

# Add Labels
df_vader['vader_label'] = np.where(df_vader['compound'] > 0, 'Positive','Negative')
criterion = df_vader['compound'].map(lambda x: x == 0)
df_vader['vader_label'][criterion] = 'Neutral'

# Groupby Date and Company for Average compound
df_vader_score = df_vader.groupby(['datepy','company']).mean()

# Keep Date, Company & compound
df_vader_score = df_vader_score[['compound','neg','neu','pos']]
df_vader_score = df_vader_score.reset_index(['datepy','company'])
df_vader_score = df_vader_score.rename(columns={'datepy' : 'Date',
                                                'company':'Name',
                                                'compound' : 'Vader_Compound',
                                                'neg' : 'Vader_Neg',
                                                'neu': 'Vader_Neu',
                                                'pos' : 'Vader_Pos'})


## Get Vader Label Counts
df_vader_counts = df_vader[['company', 'datepy', 'vader_label']]

vader_counts = df_vader_counts.groupby(by=['company','datepy'])
vader_counts = vader_counts.vader_label.value_counts()


## Turn into DF for final integration into one big DF
vader_dict = []

for k,v in vader_counts.iteritems():
    
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['Vader_Pos_Count'] = v
    elif k[2] == 'Neutral':
        s['Vader_Neutral_Count'] = v
    else:
        s['Vader_Neg_Count'] = v

    vader_dict.append(s)
    
vader_counts = pd.DataFrame(vader_dict)
vader_counts = vader_counts.groupby(['Name','Date']).sum()
vader_counts = vader_counts.fillna(0)
vader_counts = vader_counts.reset_index(level=['Date','Name'])


C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<h2> Merge all Sentiment Datasets</h2>

In [12]:
## Merge Counts

In [13]:
df_senti_combined = pd.merge(svm_counts, mnb_counts, on=['Date','Name'], how='inner')

In [14]:
df_senti_combined = pd.merge(df_senti_combined,logit_counts, on=['Date','Name'], how='inner')

In [15]:
df_senti_combined = pd.merge(df_senti_combined,twc_counts, on=['Date','Name'], how='inner')

In [16]:
df_senti_combined = pd.merge(df_senti_combined,vader_counts, on=['Date','Name'], how='inner')

In [17]:
## Merge Scores, Ratios

In [18]:
df_senti_combined = pd.merge(df_senti_combined,df_tweetwordcount_ratio, on=['Date','Name'], how='inner')

In [19]:
df_senti_combined = pd.merge(df_senti_combined,df_vader_score, on=['Date','Name'], how='inner')

In [631]:
# For Rolling Averages and Sums over 2 days - Not Implemented in Final Dataset
'''
df_test = pd.DataFrame()

for i in df_senti_combined.Name.unique():
    partial_df = df_senti_combined[df_senti_combined.Name == i].rolling(2).sum()
    df_test = df_test.append(partial_df,ignore_index=True)
    
df_senti_combined = df_test
df_senti_combined.dropna()

'''

# For volatility over N days - Not Implemented in Final Dataset
'''
df_price_test = pd.DataFrame()

for i in df_prices.Name.unique():
    partial_df = df_prices[df_prices.Name == i]
    partial_df['volatility'] = pd.rolling_std(partial_df['return'],window=2)
    df_price_test = df_price_test.append(partial_df,ignore_index=True)
    '''

"\ndf_price_test = pd.DataFrame()\n\nfor i in df_prices.Name.unique():\n    partial_df = df_prices[df_prices.Name == i]\n    partial_df['volatility'] = pd.rolling_std(partial_df['return'],window=2)\n    df_price_test = df_price_test.append(partial_df,ignore_index=True)\n    "

In [20]:
## Shift date by +1 to merge with price data of next day
df_senti_combined['Date'] = df_senti_combined.Date.map(lambda x: x + timedelta(days=1))

<h2> Combine Sentiment and Price </h2>

In [49]:
df_combined = pd.merge(df_prices,df_senti_combined, on=['Date','Name'], how='outer')

In [50]:
df_combined = df_combined.dropna(how='any')
df_combined.shape

(807, 36)

In [725]:
df_combined['Total Tweets'] = df_combined['SVM_Neg_Count'] + df_combined['SVM_Pos_Count']

In [727]:
df_combined = df_combined[['Volume','SVM_Neg_Count', 'SVM_Pos_Count', 'MNB_Neg_Count', 'MNB_Pos_Count',
       'Logit_Neg_Count', 'Logit_Pos_Count', 'TWC_Neg_Count',
       'TWC_Neutral_Count', 'TWC_Pos_Count', 'Vader_Neg_Count',
       'Vader_Neutral_Count', 'Vader_Pos_Count', 'WordCountRatio',
       'Vader_Compound', 'Vader_Neg', 'Vader_Neu', 'Vader_Pos','prev_day','avg_ret','Total Tweets',
       'direction']]

In [637]:
# See Histogram of Each Variable
'''
v_features = df_combined.ix[:,1:19].columns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

plt.figure(figsize=(12,18*4))
gs = gridspec.GridSpec(18, 1)
for i, cn in enumerate(df_combined[v_features]):
    ax = plt.subplot(gs[i])
    sns.distplot(df_combined[cn][df_combined.class_num_label == 1], bins=50)
    sns.distplot(df_combined[cn][df_combined.class_num_label == 0], bins=50)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()
'''

"\nv_features = df_combined.ix[:,1:19].columns\nimport matplotlib.pyplot as plt\nimport matplotlib.gridspec as gridspec\nimport seaborn as sns\n\nplt.figure(figsize=(12,18*4))\ngs = gridspec.GridSpec(18, 1)\nfor i, cn in enumerate(df_combined[v_features]):\n    ax = plt.subplot(gs[i])\n    sns.distplot(df_combined[cn][df_combined.class_num_label == 1], bins=50)\n    sns.distplot(df_combined[cn][df_combined.class_num_label == 0], bins=50)\n    ax.set_xlabel('')\n    ax.set_title('histogram of feature: ' + str(cn))\nplt.show()\n"

In [21]:
# Get Previous Day Direction as well 

df_combined_prevdaydirection = pd.DataFrame()

for i in df_prices.Name.unique():
    partial_df = df_prices[df_prices.Name == i]
    partial_df['prev_day'] = partial_df['direction']
    partial_df['prev_day'] = partial_df['direction'].shift(periods=1)
    partial_df['avg_ret'] = partial_df['return'].rolling(3).mean()
    df_combined_prevdaydirection = df_combined_prevdaydirection.append(partial_df,ignore_index=True)

C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [ ]:
# Different CSV for Each Company - NOT INCLUDED IN FINAL 
'''
count = 0

for i in df_combined.Name.unique():
    
    df = df_combined[df_combined['Name'] == i]
    
    total_tweets = df['Total Tweets'].sum()
    
    df = df[['SVM_Neg_Count', 'SVM_Pos_Count', 'WordCountRatio',
       'Vader_Compound', 'Vader_Neg', 'Vader_Neu', 'Vader_Pos','direction']]
    
    count = count + 1
    filename = 'By_Company/' + str(int(total_tweets)) + '_' + str(count) + str(i) + '.csv'
    
    df.to_csv(filename,index=False)
    '''

In [729]:
## Write to CSV for testing in WEKA
filename = 'Final_2ndC_PrevDayRet_AvgRet'

df_combined.to_csv(filename + '_ALL.csv', float_format='%.5f',index=False)


df_combined_20 = df_combined[df_combined['Total Tweets'] > 20]
#df_combined_20 = df_combined_20.drop('Total Tweets',axis=1)
df_combined_20.to_csv(filename + '_MoreThan20.csv', float_format='%.5f',index=False)


df_combined_50 = df_combined[df_combined['Total Tweets'] > 50]
#df_combined_50 = df_combined_50.drop('Total Tweets',axis=1)
df_combined_50.to_csv(filename + '_MoreThan50.csv', float_format='%.5f',index=False)
'''
df_combined_ONLYSNAP = df_combined[df_combined['Name'] == 'SNAP']
df_combined_ONLYSNAP = df_combined_ONLYSNAP.drop('Total Tweets',axis=1)
df_combined_ONLYSNAP.to_csv('Final_3rdClassifier_OnlySnap.csv', float_format='%.5f',index=False)

df_combined_NOSNAP = df_combined[df_combined['Name'] != 'SNAP']
df_combined_NOSNAP = df_combined_NOSNAP.drop('Total Tweets',axis=1)
df_combined_NOSNAP.to_csv('Final_3rdClassifier_OnlySnap.csv', float_format='%.5f',index=False)

'''

"\ndf_combined_ONLYSNAP = df_combined[df_combined['Name'] == 'SNAP']\ndf_combined_ONLYSNAP = df_combined_ONLYSNAP.drop('Total Tweets',axis=1)\ndf_combined_ONLYSNAP.to_csv('Final_3rdClassifier_OnlySnap.csv', float_format='%.5f',index=False)\n\ndf_combined_NOSNAP = df_combined[df_combined['Name'] != 'SNAP']\ndf_combined_NOSNAP = df_combined_NOSNAP.drop('Total Tweets',axis=1)\ndf_combined_NOSNAP.to_csv('Final_3rdClassifier_OnlySnap.csv', float_format='%.5f',index=False)\n\n"

In [68]:
## Insert into MongoDB (Needs Date converstion to Datetime)
#df_combined.Date = df_combined.Date.map(lambda t: datetime(t.year, t.month, t.day))
#FinalDataset.insert_many(df_combined.to_dict('records'))

In [108]:
WordVector = pd.read_csv('tweetwordvector.csv')

In [109]:
WordVector = WordVector.rename(columns={'Price_Date' : 'Date', 'IPO_Company' : 'Name'})

In [110]:
WordVector['Date'] = WordVector['Date'].map(lambda x: pd.to_datetime(x).date())

In [119]:
df_New = pd.merge(df_New,df_combined, on=['Name','Date'], how='outer')

In [113]:
df_New = df_New.dropna(how='any')

In [126]:
df_New.to_csv('LastTry.csv',float_format='%.5f',index=False)

Index(['Adj_Close_x', 'Adj_High_x', 'Adj_Low_x', 'Adj_Open_x', 'Adj_Volume_x',
       'Close_x', 'Date', 'Dividend_x', 'High_x', 'Low_x', 'Name', 'Open_x',
       'Split_x', 'UID_x', 'Volume_x', '_id_x', 'class_num_label_x',
       'direction_x', 'return_x', 'prev_day', 'avg_ret', 'Stock_Direction',
       'Stock_Volume', 'back', 'bold', 'capital', 'close', 'first', 'free',
       'going', 'good', 'great', 'hot', 'industrial', 'innovative', 'jagged',
       'last', 'like', 'live', 'make', 'new', 'nice', 'open', 'outperform',
       'right', 'see', 'short', 'smart', 'top', 'Adj_Close_y', 'Adj_High_y',
       'Adj_Low_y', 'Adj_Open_y', 'Adj_Volume_y', 'Close_y', 'Dividend_y',
       'High_y', 'Low_y', 'Open_y', 'Split_y', 'UID_y', 'Volume_y', '_id_y',
       'class_num_label_y', 'direction_y', 'return_y', 'SVM_Neg_Count',
       'SVM_Pos_Count', 'MNB_Neg_Count', 'MNB_Pos_Count', 'Logit_Neg_Count',
       'Logit_Pos_Count', 'TWC_Neg_Count', 'TWC_Neutral_Count',
       'TWC_Pos_Count', 'Va

In [124]:
df_New['avg_ret']

0      0.052379
1      0.010391
2     -0.005196
3      0.021613
4     -0.010849
5     -0.008409
6      0.021058
7      0.003749
8     -0.049000
9      0.003980
10     0.004660
11     0.012623
12     0.015495
13     0.010773
14     0.007090
15    -0.001507
16     0.006311
17     0.010920
18    -0.002439
19    -0.005339
20    -0.003375
21     0.009053
22    -0.005394
23    -0.032840
24     0.003202
25     0.003202
26     0.003928
27    -0.001986
28    -0.014908
29     0.001323
         ...   
930         NaN
931         NaN
932         NaN
933         NaN
934         NaN
935         NaN
936         NaN
937         NaN
938         NaN
939         NaN
940         NaN
941         NaN
942         NaN
943         NaN
944         NaN
945         NaN
946         NaN
947         NaN
948         NaN
949         NaN
950         NaN
951         NaN
952         NaN
953         NaN
954         NaN
955         NaN
956         NaN
957         NaN
958         NaN
959         NaN
Name: avg_ret, dtype: fl